In [6]:
# Initialise some variables and functions to be used
R.<x, y> = QQ[]
prec = 53
F2 = FiniteField(2)
from sage.groups.perm_gps.partn_ref.refinement_graphs import get_orbits
from collections import Counter
from time import time

# Set the curves to calculate the orbits of characteristics on
fs = [x^3*y + y^3 + x, y^3 - x*(x^4 + 3*x^2 + 1)]

# load("../../../../../sage-9.4/src/sage/schemes/riemann_surfaces/riemann_surface.py")
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface

# Set consider_subnormals to True if calculate the orbit decomposition under subnormal groups
consider_subnormals = True

# Set use_maple to True if using Maple to test if the curve is hyperelliptic and to use
# Maple to compute the differentials on the curve.
use_maple = False

# time_message is used to report the time that different steps in the algorithm take
# set verbose=False to remove this. 
def time_message(start_time, message, verbose=True):
    if verbose:
        T = time()-start_time
        if T<60:
            print("Time to "+message+": {} (seconds)".format(T))
        elif T<60^2:
            print("Time to "+message+": {} (minutes)".format(T/60))
        else:
            print("Time to "+message+": {} (hours)".format(T/3600))
        
# Helper functions to be used in computing the orbits:
# n2v turns a number n into the corresponding binary vector
def n2v(n):
    return vector(Integer(n).digits(2, padto=2*g), F2)

# v2n turns a binary vector into a number.
def v2n(v):
    return ZZ(list(v), base=2)

# permutation_from_isomorphism takes a (binary) matrix corresponding to an element of the 
# rational representation mod 2 and computed the corresponding permutation of integers [0 .. 2^(2*g)-1]
def permutation_from_isomorphism(M):
    M = M.transpose()
    Jg = block_matrix([[matrix.zero(g), matrix.identity(g)], 
                       [-matrix.identity(g), matrix.zero(g)]])
    V = vector([sum([M[i,j]*M[i,k]*Jg[j,k] for j in range(2*g) for k in range(j, 2*g)]) 
                     for i in range(2*g)])
    return [v2n(M*n2v(n)+V) for n in range(2^(2*g))]

# orbits_to_counter takes the orbits under the action of a permutation group and returns 
# a counter of the number of orbits of a given size. 
def orbits_to_counter(perms):
    co = Counter()
    ce = Counter()
    for orbit in perms:
        if parity(n2v(orbit[0])):
            co[len(orbit)] += 1
        else:
            ce[len(orbit)] += 1
    print("{} =".format(2^(g-1)*(2^g-1)), [r"{}_{}".format(a, co[a]) 
                                           for a in sorted(list(co.keys()))])
    print("{} =".format(2^(g-1)*(2^g+1)), [r"{}_{}".format(a, ce[a]) 
                                           for a in sorted(list(ce.keys()))])
    return co, ce

# parity computes the parity of a binary vector. 
def parity(v):
    return sum([v[i]*v[i+g] for i in range(g)])

if use_maple:
    try:
        MP
    except NameError:
        MP = Maple()
    MP.with_package('algcurves')
        
for f in fs:
    # The period matrix is cached so we can initialise it separately to time this burden
    ct = time()
    if use_maple:
        print("f:", f, MP('is_hyperelliptic({}, x, y)'.format(f)))
        diffs = MP('differentials({}, x, y, skip_dx)'.format(f))
        B = [R(d) for d in diffs]
        g = len(B)
        S = RiemannSurface(f, prec=prec, differentials=B)
    else:
        print("f:", f)
        S = RiemannSurface(f, prec=prec)
        g = S.genus
    S.period_matrix()
    time_message(ct, "initialise curve and cache period matrix")
    
    # We time specifically the calculation of the homomorphism basis, which is the first LLL
    ct = time()
    HB = S.homomorphism_basis(other=S)
    time_message(ct, "calculate homomorphism basis")
    
    # We time the calculation of the isomorphisms, using the hom basis we set just before
    # which is essentially timing Finke-Pohst
    ct = time()
    Mlist = S.symplectic_isomorphisms(hom_basis=HB)
    # As the action on binary vectors depends only on the value of the rational rep mod 2, we only
    # require one of M and -M in the list of isomorphism of the Jacobian. 
    Mlist_trim = [Mlist[j] for j in set([min(i, Mlist.index(-Mlist[i])) for i in range(len(Mlist))])]
    time_message(ct, "calculate isomorphisms from homomorphisms")
    
    if len(Mlist_trim)==1:
        print("Found automorphism group is trivial, investigate this case.")
        print("Observed orbits are thus trivial.")
        print()
        continue
    
    # Take a list of possible generators of the matrix group mod 2. 
    ct = time()
    possible_gens = [M.change_ring(F2) for M in Mlist_trim if not (M.is_one() or (-M).is_one())]
    moments_dict = dict()

    # create a dictionary of the characteristic polynomials of the possible generators. 
    # This will provide a general purpose way of trimming the list of possible generators
    for A in possible_gens:
        moments_dict.update({tuple(A.charpoly().list()): A})
    time_message(ct, "create moment dict")

    candidate_gens = list(moments_dict.values())
    CG = MatrixGroup(candidate_gens)

    if CG.order() >= len(Mlist_trim):
        generators = candidate_gens
    else:
        generators = Mlist_trim
        print("warning: falling back to slow implementation with massive overcounting of generators")
 
    # Get the permutation group elements from the rational rep
    ct = time()
    SS_Perms_exact = [permutation_from_isomorphism(M) 
                      for M in generators]
    time_message(ct, "realise isomorphism as permutations of characteristics")

    # get the orbits from the corresponding permutations
    ct = time()
    SS_orbits_exact = get_orbits(SS_Perms_exact, 2^(2*g))
    time_message(ct, "calculate orbits")
    
    # get the orbits
    print("Orbit decomposition:")
    co, ce = orbits_to_counter(SS_orbits_exact)
    ti = co[1]+ce[1]
    print("Total number of invariants:", ti)
    
    # Find the automorphism group of the curve as a permutation group subgroup of S_{2^{2g}}
    ct = time()
    GP = PermutationGroup([[pi+1 for pi in perm] for perm in SS_Perms_exact])
    time_message(ct, "consruct permutation group from generators")

    # Get the structure description
    ct = time()
    SD = GP.structure_description()
    time_message(ct, "get structure description")
    print("Structure description (reduced automorphism group):", SD)
    if ':' in SD:
        print("Group ID:", GP.group_id())
    
    oddi = ', '.join(["${}_{{{}}}$".format(a, co[a]) 
                       for a in sorted(list(co.keys()))])
    eveni = ', '.join(["${}_{{{}}}$".format(a, ce[a]) 
                       for a in sorted(list(ce.keys()))])

    # If there is 1 invariant look at the orbit decomposition of the subnormals
    if not ((ti == 1) and consider_subnormals):
        print()
        continue
        
    # Find the subgroups, and make a poset by H <= K if H is a normal subgroup of K
    subgroups = GP.subgroups()
    GPoset = Poset((subgroups, lambda h, k: h.is_subgroup(k) and h.is_normal(k)))
    # Make a list of all subnormal groups by following down the poset
    GIP = subgroups[-1]
    subnormals = set([])
    extra = GPoset.lower_covers(GIP)
    while extra:
        subnormals.update(extra)
        extra = set(sum([GPoset.lower_covers(H) for H in extra], []))
    # We exclude the last element of the list as we know it will be the identity subnormal,
    # which will be uninteresting for our considerations.
    subnormals.remove(subgroups[0])
    subnormals = list(subnormals)
    if subnormals:
        print("Considering subnormals:")

    # Print the orbit decomposition for the subnormals
    for H in subnormals:
        subgroup_orbits = get_orbits([[ii-1 for ii in gi.tuple()] 
                                      for gi in H.gens()], 2^(2*g))
        print("Structure description (subnormal group):", H.structure_description())
        print("Orbit decomposition:")
        _ = orbits_to_counter(subgroup_orbits)

    print()

f: x^3*y + y^3 + x
Time to initialise curve and cache period matrix: 1.1022732257843018 (seconds)
Time to calculate homomorphism basis: 0.7451262474060059 (seconds)
Time to calculate isomorphisms from homomorphisms: 0.10796380043029785 (seconds)
Time to create moment dict: 0.0340418815612793 (seconds)
Time to realise isomorphism as permutations of characteristics: 0.02764725685119629 (seconds)
Time to calculate orbits: 7.033348083496094e-05 (seconds)
Orbit decomposition:
28 = ['28_1']
36 = ['1_1', '7_2', '21_1']
Total number of invariants: 1
Time to consruct permutation group from generators: 0.0010788440704345703 (seconds)
Time to get structure description: 0.007700920104980469 (seconds)
Structure description (reduced automorphism group): PSL(3,2)

f: -x^5 - 3*x^3 + y^3 - x
Time to initialise curve and cache period matrix: 1.545882225036621 (seconds)
Time to calculate homomorphism basis: 2.231006622314453 (seconds)
Time to calculate isomorphisms from homomorphisms: 0.02076005935668945